In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from state import get_state, queue
from basic.reward_basic import calculate_reward
from basic.learner_basic import Learner
from traffic_lights_maps import get_traffic_light_phases, make_map
import os
import sys

import traci
import random
import gym
from gym import spaces

In [3]:
from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
import numpy as np
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt

In [4]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [36]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(32, activation="elu", input_shape=input_shape),
    layers.Dense(32, activation="elu"),
    # layers.Dense(2, activation="elu"),
    layers.Dense(n_outputs)])

/Users/arseneclaustre/.pyenv/versions/3.10.6/envs/RL_traffic/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [6]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [7]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Nadam(learning_rate=1e-1)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [17]:
import traci

class SimulationSUMO:
    def __init__(self, sumoCmd,frame):
        self.frame=frame
        if traci.isLoaded():
            traci.close()
        traci.start(sumoCmd)  # Start SUMO once

    def step(self,action):
        ###CODER UN STEP qui prend une action en argument
        #utiliser un modele, renvoyer next state: array, reward:int, done :

        state = np.array(self.queue(self.get_lane()))
        traci.trafficlight.setPhase(self.get_trafficlight_id()[0],2*action)
        for _ in range(self.frame):
            traci.simulationStep()
        next_state = np.array(self.queue(self.get_lane()))
        reward = sum(next_state)-sum(state)
        return next_state,reward


    def get_state(self):
        return traci.vehicle.getIDList()  # Example: Get list of vehicles

    def get_lane(self):
        return traci.lane.getIDList()

    def get_trafficlight_id(self):
        return traci.trafficlight.getIDList()

    def queue(self,lane_ids):
        return [traci.lane.getLastStepHaltingNumber(lane_id) for lane_id in lane_ids]
    def close(self):
        traci.close()  # Properly close SUMO
        os.system("pkill -f sumo")

# Example usage
# sumoCmd = [sumoBinary2, "-c", sumoConfig3, "--start"]
# simulation = SimulationSUMO(sumoCmd)

# for _ in range(10000):  # Run 100 steps
#     simulation.step()
#     print(simulation.get_state())  # Print vehicle IDs

# simulation.close()  # Close SUMO properly

In [22]:
sumoBinary = "/Users/arseneclaustre/sumo/bin/sumo-gui"
sumoBinary2 = "/Users/arseneclaustre/sumo/bin/sumo"
sumoConfig = r"C:/Program Files/rl_project/2025-02-12-16-24-37/osm.sumocfg"
sumoConfig2 = r"C:/Program Files/rl_project/2025-02-25-19-05-44/osm.sumocfg"
# sumoConfig3 = r"/home/psels/code/psels/RL_traffic/RL_traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"


rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=2000)

sumoCmd = [sumoBinary2, "-c", sumoConfig3, "--start"]
for episode in range(300):
    print(f'episode : {episode}')
    epsilon = max(1 - episode / 150, 0.01)
    simulation=SimulationSUMO(sumoCmd,500)
    lane_ids = simulation.get_lane()
    # print(lane_ids[0])

    #trafic_light_ids = traci.trafficlight.getIDList()
    trafic_light_ids = simulation.get_trafficlight_id()

    state = np.array(simulation.queue(lane_ids))

    action=-1
    wait_times.append(0)
    for _ in range(20): ## TO CHANGED
        action = epsilon_greedy_policy(state, epsilon)
        next_state,reward = simulation.step(action)
        replay_buffer.append((state, action, reward, next_state))
        state = next_state
        if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)

    simulation.close()

episode : 0
 Retrying in 1 seconds


Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 0 BUF 0)                     
episode : 1
 Retrying in 1 seconds


Step #100.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 33 ACT 2 BUF 0)                     
episode : 2
 Retrying in 1 seconds


state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]         
Q_values : [ 0.13195437  0.63871884  0.22971821 -0.7547903 ]
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 33 ACT 10 BUF 0)                    
episode : 3
 Retrying in 1 seconds


Step #100.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 33 ACT 0 BUF 0)                     
episode : 4
 Retrying in 1 seconds


...

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 1.5291102  3.3053164  2.6625113 -3.9018111]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 3.419117   2.6033113  2.7814834 -3.153415 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 2.7607756  2.625033   2.8886118 -2.479518 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 2.6315188  2.4815636  2.8327336 -1.8727093]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 2.4442096  2.2704363  2.7667422 -1.326309 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 1.9795921  2.8352826  2.8140655 -1.313793 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 2.355811   2.6625574  2.8258271 -1.0753498]
state : [0 0 0 0 0 0 0 0 0 

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 5 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 3.109989    2.6078339   2.7755256  -0.49322653]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 6 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 2.495144    1.7684361   2.7681649  -0.33647013]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 3.1661086   2.6784768   2.6801946  -0.01474357]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [2.6461403 3.1765015 2.6292372 0.1893754]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [2.6008477  2.645214   2.2615926  0.37317443]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [2.553114  2.8259025 1.2508723 0.5386739]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [3.0852165  2.994123   0.7363217  0.68769765]
state : [0 0 0 0 0 0 0 

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 3.2195804  1.6761838 -1.8576808  1.2377133]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 2.320626    1.9436551  -0.12003303  1.3171513 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 1.9274487   1.5608901  -0.13100982  1.3886814 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [1.3398191 1.9520084 0.2972648 1.4530871]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [1.7184274 2.2195728 0.6280596 1.5110848]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [2.8144054  2.1448894  0.92592573 0.4655516 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [2.007677   2.0377965  1.1941383  0.04431629]
state : [0 0 0 0 0 0 0 0 0 

Interrupt signal received, trying to exit gracefully.


KeyboardInterrupt: 

In [25]:
sumoBinary = "/Users/arseneclaustre/sumo/bin/sumo-gui"
sumoBinary2 = "/Users/arseneclaustre/sumo/bin/sumo"
sumoConfig = r"C:/Program Files/rl_project/2025-02-12-16-24-37/osm.sumocfg"
sumoConfig2 = r"C:/Program Files/rl_project/2025-02-25-19-05-44/osm.sumocfg"
# sumoConfig3 = r"/home/psels/code/psels/RL_traffic/RL_traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"


rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=2000)


sumoCmd = [sumoBinary, "-c", sumoConfig3, "--start"]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])

trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(queue(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(10000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    state=np.array(queue(lane_ids))
    if step%1000 == 0:
        state=np.array(queue(lane_ids))
        action = epsilon_greedy_policy(state,0)
        print("action", action*2)
        traci.trafficlight.setPhase(trafic_light_ids[0],action*2)
    traci.simulationStep()

traci.close()

 Retrying in 1 seconds
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [  2.2881737   3.4344368  -0.403687  -11.053476 ]
action 2
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [  2.2881737   3.4344368  -0.403687  -11.053476 ]
action 2
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [  2.2881737   3.4344368  -0.403687  -11.053476 ]
action 2
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [  2.2881737   3.4344368  -0.403687  -11.053476 ]
action 2
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [  2.288175     3.4344344   -0.40368724 -11.053474  ]
action 2
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 6 6 0 0 0 0 2 1 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [  2.288175     3.4344335   -0.40368652 -11.053474  ]
action 2
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 6 6 0 0 0 0 4 4 

FatalTraCIError: Connection closed by SUMO.